In [4]:
import pandas as pd
import numpy as np
import gspread
from pydata_google_auth import get_user_credentials
from google.cloud import bigquery
import re
import os


project_id = 'ledger-fcc1e'
creds = get_user_credentials([
    'https://www.googleapis.com/auth/cloud-platform', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
])
client = bigquery.Client(project=project_id, credentials=creds)

gc = gspread.authorize(creds) 
pull_data = lambda query: client.query(query).result().to_dataframe()
get_worksheet = lambda key, name: gc.open_by_key(key).worksheet(name)

In [5]:
### Get data to be graded
# query =\
# '''
# SELECT 
#     phone_number_clean,
#     ms_area,
#     interested_to_edc_flag AS interest,
#     acquisition_channel,
#     user_age,
#     kyc_tier,
#     loyalty_tier,
#     age_on_core_days,
#     age_ncore_days,
#     age_on_accounting_days,
#     referee_count,
#     est_daily_customer,
#     count_trf,
#     edc_count,
#     edc_type,
#     business_age,
#     core_before_shutdown_flag,
#     ppob_before_shutdown_flag,
#     M0_core_count,
#     M0_ppob_payment_count,
#     M0_QRIS_payment_count,
#     LM_core_count,
#     LM_ppob_payment_count,
#     LM_QRIS_payment_count,
#     L2M_core_count,
#     L2M_ppob_payment_count,
#     L2M_QRIS_payment_count,
#     L3M_core_count,
#     L3M_ppob_payment_count,
#     L3M_QRIS_payment_count,
#     M0_corepayment_TPV,
#     M0_ppob_TPV,
#     M0_QRIS_TPV,
#     LM_corepayment_TPV,
#     LM_ppob_TPV,
#     LM_QRIS_TPV,
#     L2M_corepayment_TPV,
#     L2M_ppob_TPV,
#     L2M_QRIS_TPV,
#     L3M_corepayment_TPV,
#     L3M_ppob_TPV,
#     L3M_QRIS_TPV,
#     m0_transaxi_cnt,
#     m0_Utang_cnt,
#     lm_transaxi_cnt,
#     lm_Utang_cnt,
#     m2_transaxi_cnt,
#     m2_Utang_cnt,
#     m3_transaxi_cnt,
#     m3_Utang_cnt,
# FROM `ledger-fcc1e.dg_buku_trnsfmd.EDC_BW_userbase_all`
# WHERE interested_to_edc_flag = '-1'
# '''
# raw = client.query(query).result().to_dataframe()
# raw.to_pickle('data/deploy_raw.pkl')
# raw = pd.read_pickle('data/deploy_raw.pkl')
# raw.info()

In [6]:
query =\
'''
SELECT *
FROM `ledger-fcc1e.temp_ryan.edc_grading_deploy`
'''

raw = client.query(query).result().to_dataframe()
raw.to_pickle('data/deploy_raw.pkl')
raw = pd.read_pickle('data/deploy_raw.pkl')
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321373 entries, 0 to 321372
Data columns (total 59 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   phone_number_clean         321373 non-null  object
 1   ms_area                    269201 non-null  object
 2   interest                   321373 non-null  object
 3   acquisition_channel        321373 non-null  object
 4   user_age                   127414 non-null  Int64 
 5   kyc_tier                   321373 non-null  object
 6   loyalty_tier               321373 non-null  object
 7   age_on_core_days           52169 non-null   Int64 
 8   age_ncore_days             23635 non-null   Int64 
 9   age_on_accounting_days     278972 non-null  Int64 
 10  referee_count              6616 non-null    Int64 
 11  est_daily_customer         8564 non-null    object
 12  count_trf                  11001 non-null   object
 13  edc_count                  8645 non-null    

In [7]:
# Clean data
lq = [col for col in raw.columns if col.startswith('LQ')]
# acc = [col for col in raw.columns if col.endswith('cnt')]

df =\
(
    raw
    .assign(
        # interested_to_EDC_flag = lambda x: (pd.to_numeric(x.interested_to_EDC_flag)).astype(bool),
        # is_core_q0 = lambda x: np.where(pd.isnull(x.LQ0_core_count), 0, 1).astype(bool),
        is_core_q1 = lambda x: np.where(pd.isnull(x.LQ1_core_count), 0, 1).astype(bool),
        is_core_q2 = lambda x: np.where(pd.isnull(x.LQ2_core_count), 0, 1).astype(bool),
        is_core_q3 = lambda x: np.where(pd.isnull(x.LQ3_core_count), 0, 1).astype(bool),
        # is_ppob_q0 = lambda x: np.where(pd.isnull(x.LQ0_ppob_count), 0, 1).astype(bool),
        is_ppob_q1 = lambda x: np.where(pd.isnull(x.LQ1_ppob_count), 0, 1).astype(bool),
        is_ppob_q2 = lambda x: np.where(pd.isnull(x.LQ2_ppob_count), 0, 1).astype(bool),
        is_ppob_q3 = lambda x: np.where(pd.isnull(x.LQ3_ppob_count), 0, 1).astype(bool),
        # is_qris_q0 = lambda x: np.where(pd.isnull(x.LQ0_qris_count), 0, 1).astype(bool),
        is_qris_q1 = lambda x: np.where(pd.isnull(x.LQ1_qris_count), 0, 1).astype(bool),
        is_qris_q2 = lambda x: np.where(pd.isnull(x.LQ2_qris_count), 0, 1).astype(bool),
        is_qris_q3 = lambda x: np.where(pd.isnull(x.LQ3_qris_count), 0, 1).astype(bool),
        # is_core_q4 = lambda x: np.where(x.LQ4_core_count > 0, 1, 0).astype(bool),
        # is_done_payment_flag = lambda x: np.where(pd.isnull(x.L3M_core_count), 0, 1).astype(bool),
        est_daily_customer = lambda x: pd.to_numeric(x.est_daily_customer, errors='coerce'),
        count_trf = lambda x: pd.to_numeric(x.count_trf),
        edc_count = lambda x: x.edc_count.str.replace('-','to'),
        edc_type = lambda x: x.edc_type.fillna('Empty').str.split(','),
        acquisition_channel = lambda x: x.acquisition_channel.str.replace(' Acquisition',''),
        business_age = lambda x: x.business_age.str.replace('-','to').str.split(',').str[0],
        core_before_shutdown_flag = lambda x: x.core_before_shutdown_flag.astype(bool),
        ppob_before_shutdown_flag = lambda x: x.ppob_before_shutdown_flag.astype(bool),
        kyc_tier = lambda x: x.kyc_tier.replace('NON-KYC','NONKYC').fillna('NONKYC'),
        loyalty_tier = lambda x: x.loyalty_tier.fillna('Bronze'),
        age_on_core_days = lambda x: x.age_on_core_days.fillna(0),
        age_on_ncore_days = lambda x: x.age_ncore_days.fillna(0),
        # age_on_accounting_days = lambda x: x.age_on_accounting_days.fillna(0)
    )
    .drop(columns=['user_age', 'referee_count', *lq])
)

# change the TPV features to numeric
tpv_col = df.filter(regex='TPV|tpv').columns
for i in tpv_col:
    df[i] = pd.to_numeric(df[i])

# normalize the numeric columns
num_col = [col for col in df.select_dtypes(include=['float', 'int']).columns if '_flag' not in col]
num_col = [col for col in df[num_col].columns if 'age' not in col]
for i in num_col:
    df[i] = df[i].fillna(0).apply(np.log1p)

# one-hot encode the edc_type columns
for i in sorted(set(sum(df['edc_type'].tolist(),[]))):  
    df[f'edc_type_{i}'] = (df['edc_type'].apply(lambda x: x.count(i) if i in x else None)).fillna(0).astype(bool)

# one-hot encode other columns
df = pd.get_dummies(data=df, columns=['ms_area', 'acquisition_channel', 'kyc_tier', 'loyalty_tier', 'edc_count', 'business_age'], prefix=['ms', 'acq', 'kyc', 'lyl', 'edc_count', 'bizage'], dtype=bool)
# df = pd.get_dummies(data=df, columns=['ms_area', 'acquisition_channel', 'kyc_tier', 'loyalty_tier'], prefix=['ms', 'acq', 'kyc', 'lyl'], dtype=bool)



# drop some columns
df = df.drop(columns=['edc_type_Empty', 'edc_type', 'age_ncore_days', 'kyc_NONKYC'])
# df = df.drop(columns=['age_ncore_days'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321373 entries, 0 to 321372
Data columns (total 88 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   phone_number_clean         321373 non-null  object 
 1   interest                   321373 non-null  object 
 2   age_on_core_days           321373 non-null  Int64  
 3   age_on_accounting_days     278972 non-null  Int64  
 4   est_daily_customer         321373 non-null  float64
 5   count_trf                  321373 non-null  float64
 6   core_before_shutdown_flag  321373 non-null  bool   
 7   ppob_before_shutdown_flag  321373 non-null  bool   
 8   m0_transaxi_cnt            321373 non-null  Float64
 9   m0_Utang_cnt               321373 non-null  Float64
 10  lm_transaxi_cnt            321373 non-null  Float64
 11  lm_Utang_cnt               321373 non-null  Float64
 12  m2_transaxi_cnt            321373 non-null  Float64
 13  m2_Utang_cnt               32

In [9]:
bucket = []
for i in pd.read_csv('result/probability_table.csv').score_bucket.unique():
    bucket.extend(i.replace('(','').replace(']', '').split(', '))
bucket = list(map(float,sorted(set(bucket))))
bucket

[-0.001, 0.01, 0.02, 0.03, 0.05, 0.08, 0.13, 0.22, 0.44, 0.85, 1.0]

In [10]:
from pycaret.classification import *
from pycaret.datasets import get_data
import pickle
from datetime import date
from itertools import chain
from collections import Counter
from git import Repo

def apply_randoms(x):
    key = x['score_bucket'].iat[0]
    return pd.Series(np.random.choice(['Control', 'Test'], size=len(x), p=[0.5, 0.5]))

def create_result(df, criteria=None, save=False, getAll=False):
    # filter out based on criteria
    # if criteria == None:
    #     df = df
    # else:
    #     criteria = ' & '.join([criteria])
    #     df = df.query(criteria)

    # get the buckets
    bucket = []
    for i in pd.read_csv('result/probability_table.csv').score_bucket.unique():
        bucket.extend(i.replace('(','').replace(']', '').split(', '))
    bucket = list(map(float,sorted(set(bucket))))

    # get the model
    model = pickle.load(open('models/model.sav','rb'))

    # add some columns needed by the model to the df
    added_col = [x for x in model._feature_names_in if x not in df.columns.to_list()]
    df[added_col] = 0

    # get prediction
    pred = predict_model(model, df, raw_score=True)

    # get result
    result =\
        (
            pred
            .assign(
                score_bucket = lambda x: pd.cut(x.prediction_score_1, bins=bucket).astype(str),
                test_group = lambda x: x.groupby(['score_bucket']).apply(apply_randoms).reset_index().drop('level_1', 1).iloc[:, 1],
                grade = lambda x: np.select(
                    [x.score_bucket == f'({bucket[0]}, {bucket[1]}]', x.score_bucket == f'({bucket[1]}, {bucket[2]}]', x.score_bucket == f'({bucket[2]}, {bucket[3]}]', x.score_bucket == f'({bucket[3]}, {bucket[4]}]', x.score_bucket == f'({bucket[4]}, {bucket[5]}]', x.score_bucket == f'({bucket[5]}, {bucket[6]}]', x.score_bucket == f'({bucket[6]}, {bucket[7]}]', x.score_bucket == f'({bucket[7]}, {bucket[8]}]', x.score_bucket == f'({bucket[8]}, {bucket[9]}]', x.score_bucket == f'({bucket[9]}, {bucket[10]}]'],
                    ['J', 'I', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']
                )
            )
        )
    
    if getAll:
        result = result
    else:
        result = result[['phone_number_clean', 'score_bucket', 'test_group', 'grade']]

    if criteria is None:
        res = result.sort_values(['score_bucket', 'test_group'])
    else:
        criteria = ' & '.join([criteria])
        res = result.query(criteria).sort_values(['score_bucket', 'test_group'])

    if save:
        today = date.today().strftime("%d%m")
        modelname = Repo().head.ref.name
        res.to_csv(f'deploy_result/{modelname}_result_{today}.csv', index=False)

    return res

res = create_result(df, save=True, getAll=True)

res

,phone_number_clean,interest,age_on_core_days,age_on_accounting_days,est_daily_customer,count_trf,core_before_shutdown_flag,ppob_before_shutdown_flag,m0_transaxi_cnt,m0_Utang_cnt,...,bizage_1to3,bizage_3to5,bizage_5+,interested_to_EDC_flag,prediction_label,prediction_score_0,prediction_score_1,score_bucket,test_group,grade
9,82262436904,-1,0,40,0.0,0.0,False,False,0.0,0.0,...,False,False,False,0,0,1.00,0.00,"(-0.001, 0.01]",Control,J
21,85719787589,-1,0,152,0.0,0.0,False,False,0.0,1.94591,...,False,False,False,0,0,1.00,0.00,"(-0.001, 0.01]",Control,J
25,81337511262,-1,0,863,0.0,0.0,False,False,0.0,1.609438,...,False,False,False,0,0,1.00,0.00,"(-0.001, 0.01]",Control,J
26,81779936986,-1,0,40,0.0,0.0,False,False,0.0,2.564949,...,False,False,False,0,0,1.00,0.00,"(-0.001, 0.01]",Control,J
28,81285262945,-1,0,605,0.0,0.0,False,False,0.0,1.098612,...,False,False,False,0,0,0.99,0.01,"(-0.001, 0.01]",Control,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302958,8568991621,-1,553,531,0.0,0.0,False,False,0.0,0.0,...,False,False,False,0,1,0.10,0.90,"(0.85, 1.0]",Test,A
308760,85736533944,-1,637,1203,0.0,0.0,False,False,0.0,0.0,...,False,False,False,0,1,0.12,0.88,"(0.85, 1.0]",Test,A
308790,85281444777,-1,480,412,0.0,0.0,False,False,0.0,0.0,...,False,False,False,0,1,0.08,0.92,"(0.85, 1.0]",Test,A
321354,89530436417,-1,951,951,0.0,0.0,False,False,0.0,0.0,...,False,False,False,0,1,0.13,0.87,"(0.85, 1.0]",Test,A


In [11]:
res.score_bucket.value_counts()

(-0.001, 0.01]    72125
(0.05, 0.08]      38031
(0.22, 0.44]      34836
(0.08, 0.13]      34727
(0.13, 0.22]      32960
(0.44, 0.85]      31131
(0.01, 0.02]      30254
(0.03, 0.05]      28367
(0.02, 0.03]      17328
(0.85, 1.0]        1614
Name: score_bucket, dtype: int64

In [12]:
job_config = bigquery.LoadJobConfig(
    source_format = bigquery.SourceFormat.CSV, autodetect=True, 
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

In [14]:
file_path = "deploy_result\ds_edc_result_0808.csv"
table_id = "ledger-fcc1e.temp_ryan.edc_user_grade_all"
with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)